In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pyLDAvis
import pyLDAvis.sklearn
import pandas as pd
import jieba
df1 = pd.read_csv("dat/weibo.csv", encoding='utf-8')
df=pd.DataFrame(df1['text'].astype(str))

#加载停用词表
a=r'停用词表stopwords\746.txt'
b=r'停用词表stopwords\902.txt'
c=r'停用词表stopwords\1208.txt'
d=r'停用词表stopwords\1447.txt'
e=r'停用词表stopwords\1893.txt'
f=r'停用词表stopwords\中文.txt'
g=r'停用词表stopwords\哈工大停用词表.txt'
h=r'停用词表stopwords\四川大学机器智能实验室停用词库.txt'
i=r'停用词表stopwords\百度停用词列表.txt'
j=r'停用词表stopwords\符号.txt'
k=r'停用词表stopwords\英文.txt'
l=r'停用词表stopwords\stopwords.txt'
lis=[a,b,c,d,e,f,g,h,i,j,k,l]
def loadstop_words(path):
    # 停用词：结合多个停用词表
    stop_words = set()
    for path in lis:
        with open(path,'r',encoding='utf8') as fr:
            for line in fr:
                item = line.strip()
                stop_words.add(item)
    custom_stop_words=set()
    stop_words=stop_words | custom_stop_words
    return stop_words
stopwords=loadstop_words(lis)

#分词，并添加限制条件分词
def chinese_word_cut(text):
    s=list(jieba.cut(text))
    s1=[]
    for word in s:
        if len(word) <= 2 and word not in stopwords and not word.isdigit():
            s1.append(word)
    if not s1:
        return '空'
    else:
        return ' '.join(s1)
    
df['text_cutted']=df.text.apply(chinese_word_cut)
# print(df.text_cutted.loc[[2]])
# print(df.iloc[[1]])

n_features = 1000

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words=stopwords,
                                max_df = 0.5,
                                min_df = 5)

tf = tf_vectorizer.fit_transform(df.text_cutted)

n_topics = 20
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=50,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)


# print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
    print()

n_top_words = 15
# tf_feature_names = tf_vectorizer.get_feature_names()
# print_top_words(lda, tf_feature_names, n_top_words)
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)




FileNotFoundError: [Errno 2] File b'dat/weibo.csv' does not exist: b'dat/weibo.csv'